In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
from skimage import io
from skimage.io import imread_collection
%matplotlib inline

In [65]:
pos_dir = 'ex3_data/positives/positives/*.png'
neg_dir = 'ex3_data/negatives/negatives/*.png'


negatives = imread_collection(neg_dir)
positives = imread_collection(pos_dir)

In [71]:
plt.imshow(positives[0])

In [206]:
def extract_features(im):
    im = np.mean(im, axis=2)
    plt.imshow(im)
    hist,_ = np.histogram(im, bins=5, range=None, normed=False, weights=None, density=None)
    features = []
    
    features.append(np.mean(im))
    features.append(np.max(im))
    features.append(np.min(im))
    
    return features

In [195]:
pos_features = [extract_features(i) for i in positives]
neg_features = [extract_features(i) for i in negatives]

[160.40972222222223, 181.0, 103.0]

In [207]:
extract_features(positives[0])

In [208]:
def mean(features):
    return np.mean(features, axis=0)

In [209]:
mean_pos = mean(pos_features)
mean_neg = mean(neg_features)

In [210]:
print(mean_pos)
print(mean_neg)

[ 152.87141204  181.61111111   87.84444444]
[ 139.29417438  160.86666667  118.63333333]


In [211]:
def compute_covariance_subroutine(features_mx, mean):
    cov_mx = np.zeros((len(mean), len(mean)))
    vecs = features_mx - mean
    for vec in vecs:
        mx = np.outer(vec, vec)
        cov_mx = cov_mx + mx
    return cov_mx

def compute_covariance_matrix(features_mx1, mean1, features_mx2, mean2):
    cov_mx1 = compute_covariance_subroutine(features_mx1, mean1)
    cov_mx2 = compute_covariance_subroutine(features_mx2, mean2)
    
    return (cov_mx1 + cov_mx2)/(len(features_mx1) + len(features_mx2))

In [212]:
E = compute_covariance_matrix(pos_features, mean_pos, neg_features, mean_neg)
invE = np.linalg.inv(E)

In [213]:
def compute_probability(features, mean, inv_covariance):
    vec = features - mean
    p = np.dot(vec, inv_covariance)
    p = np.dot(p, vec)
    p = - p/2
    return np.exp(p)

In [214]:
false_negatives = 0
false_positives = 0

bad_pos = []
bad_neg = []

for i in range(len(pos_features)):
    pP = compute_probability(pos_features[i], mean_pos, invE)
    nP = compute_probability(pos_features[i], mean_neg, invE)
    
    if pP < nP:
        false_negatives += 1
        bad_neg.append(i)
        
for i in range(0, len(neg_features)):
    pP = compute_probability(neg_features[i], mean_pos, invE)
    nP = compute_probability(neg_features[i], mean_neg, invE)

    if pP > nP:
        false_positives = false_positives + 1
        bad_pos.append(i)
        
print (false_negatives)
print (false_positives)

0
0


In [215]:
plt.imshow(negatives[bad_pos[0]])

IndexError: list index out of range

In [216]:
print(invE)

[[ 0.03669633 -0.02402913 -0.01830397]
 [-0.02402913  0.02609394  0.00677018]
 [-0.01830397  0.00677018  0.01830666]]
